In [1]:
import mdtraj as md
import numpy as np
import matplotlib.pyplot as plt
import csv
import xlrd
import os
import os.path
from pathlib2 import Path
from decimal import *
%matplotlib inline
import matplotlib.pylab as pylab
params = {'legend.fontsize': 'x-large',
          'figure.figsize': (15, 5),
         'axes.labelsize': 'x-large',
         'axes.titlesize':'x-large',
         'xtick.labelsize':'x-large',
         'ytick.labelsize':'x-large',
         'font.weight':'medium', 
         'xtick.major.size':8,
         'ytick.major.size':8}
pylab.rcParams.update(params)

In [2]:
filedir = '/gpfs/amarolab/monolayers/'

In [3]:
laup_test_20 = md.load(filedir+'LAUP_NaCl0.4_GMX/SA20-1_100ns/step9_1.trr', top=filedir+'LAUP_NaCl0.4_GMX/SA20-1_100ns/step9_1.gro')
laup_test_19 = md.load('/gpfs/amarolab/abbyflabby/LAU/LAUP_NaCl0.4/SA19/charmm-gui/namd/step7.1_production.dcd', top='/gpfs/amarolab/abbyflabby/LAU/LAUP_NaCl0.4/SA19/charmm-gui/step5_assembly.xplor_ext.psf')
print(laup_test_20)
print(laup_test_19)

<mdtraj.Trajectory with 10001 frames, 42576 atoms, 7674 residues, and unitcells>
<mdtraj.Trajectory with 5000 frames, 36223 atoms, 7721 residues, and unitcells>


In [4]:
mix1_test_20 = md.load(filedir+'MIX1_NaCl0.4_GMX/SA20-3_100ns/concat.trr', top=filedir+'MIX1_NaCl0.4_GMX/SA20-3_100ns/step9_1.gro')
mix1_test_19 = md.load(filedir+'MIX1_NaCl0.4_GMX/SA19-2_100ns/newtraj.trr', top=filedir+'MIX1_NaCl0.4_GMX/SA19-2_100ns/step9_1.gro')
print(mix1_test_20)
print(mix1_test_19)

<mdtraj.Trajectory with 8001 frames, 46538 atoms, 7625 residues, and unitcells>
<mdtraj.Trajectory with 11901 frames, 47628 atoms, 7669 residues, and unitcells>


In [12]:
def get_nematic_orders_upper_chains(traj):
    traj.center_coordinates()
    #residues = [residue.index for residue in traj.top.residues if residue.name =='PALP' or residue.name == 'MYRP' or residue.name=='LAUP' or residue.name=='STEP']  
    residues = [residue.index for residue in traj.top.residues if residue.name=='LAUP']  
    atoms = []
    for i in residues:
        print(i)
        atoms.append([atom.index for atom in traj.top.atoms if atom.residue.index == residues[i] and traj.xyz[0][atom.index][2]>0])
    return(md.compute_nematic_order(traj, indices=atoms))
get_nematic_orders_upper_chains(laup_test_19)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

array([0.30461941, 0.30062407, 0.30482284, ..., 0.31994041, 0.31614836,
       0.30709799])

In [11]:
def get_nematic_orders_upper_chains(traj):
    traj.center_coordinates()
    residues = [residue.index for residue in traj.top.residues if residue.name =='PALP' or residue.name == 'MYRP' or residue.name=='LAUP' or residue.name=='STEP']  
    #residues = [residue.index for residue in traj.top.residues if residue.name=='LAUP']  
    atoms = []
    for i in residues:
        atoms.append([atom.index for atom in traj.top.atoms if atom.residue.index == residues[i] and traj.xyz[0][atom.index][2]>0])
    return(md.compute_nematic_order(traj, indices=atoms))
def get_nematic_orders_lower_chains(traj):
    traj.center_coordinates()
    residues = [residue.index for residue in traj.top.residues if residue.name=='LAUP']
    #residues = [residue.index for residue in traj.top.residues if residue.name =='PALP' or residue.name == 'MYRP' or residue.name=='LAUP' or residue.name=='STEP']
    atoms = []
    for i in residues:
        #atoms.append([atom.index for atom in traj.top.atoms if atom.residue.index == residues[i] and traj.xyz[0][atom.index][2]<=0])
        atoms.append([atom.index for atom in traj.top.atoms if atom.residue.index == residues[i] and traj.xyz[0][atom.index][2]<0])
    return(md.compute_nematic_order(traj, indices=atoms))
    #return(md.compute_nematic_order(traj, indices=atoms))
print('laup test 19')
laup_test_19_nemu = get_nematic_orders_upper_chains(laup_test_19)
print(laup_test_19_nemu)
laup_test_19_neml = get_nematic_orders_lower_chains(laup_test_19)
print(laup_test_19_neml)
print('mix test 19')
mix1_test_19_nemu = get_nematic_orders_upper_chains(mix1_test_19)
print(mix1_test_19_nemu)
mix1_test_19_neml = get_nematic_orders_lower_chains(mix1_test_19)
print(mix1_test_19_neml)
print('mix test 20')
mix1_test_20_nemu = get_nematic_orders_upper_chains(mix1_test_20)
print(mix1_test_20_nemu)
mix1_test_20_neml = get_nematic_orders_lower_chains(mix1_test_20)
print(mix1_test_20_neml)
print('laup test 20')
laup_test_20_nemu = get_nematic_orders_upper_chains(laup_test_20)
print(laup_test_20_nemu)
laup_test_20_neml = get_nematic_orders_lower_chains(laup_test_20)
print(laup_test_20_neml)

laup test 19
[0.30461941 0.30062407 0.30482284 ... 0.31994041 0.31614836 0.30709799]
[0.31086065 0.31451249 0.29586009 ... 0.32051998 0.34573805 0.3466618 ]
mix test 19


IndexError: list index out of range

In [7]:
print('mix test 19')
mix1_test_19_nemu = get_nematic_orders_upper_chains(mix1_test_19)
print(mix1_test_19_nemu)
mix1_test_19_neml = get_nematic_orders_lower_chains(mix1_test_19)
print(mix1_test_19_neml)
print('mix test 20')
mix1_test_20_nemu = get_nematic_orders_upper_chains(mix1_test_20)
print(mix1_test_20_nemu)
mix1_test_20_neml = get_nematic_orders_lower_chains(mix1_test_20)
print(mix1_test_20_neml)
print('laup test 20')
laup_test_20_nemu = get_nematic_orders_upper_chains(laup_test_20)
print(laup_test_20_nemu)
laup_test_20_neml = get_nematic_orders_lower_chains(laup_test_20)
print(laup_test_20_neml)

mix test 19
[0.29923519 0.29596482 0.28432967 ... 0.29004568 0.29705604 0.29436516]
[0.29164688 0.3032263  0.29593163 ... 0.30440304 0.30656066 0.28956421]
mix test 20
[0.31460835 0.32200992 0.32094884 ... 0.32198896 0.30796172 0.31574381]
[0.30954457 0.30949765 0.30181467 ... 0.2880521  0.28479709 0.28550938]
laup test 20
[0.32145591 0.32562976 0.32527333 ... 0.32138582 0.33607734 0.32024034]
[0.34536806 0.3796297  0.37005913 ... 0.41506077 0.42810748 0.43641232]


In [ ]:
mix1_1 = md.load(filedir+'MIX1_NaCl0.4_GMX/SA20-3_100ns/concat.trr', top=filedir+'MIX1_NaCl0.4_GMX/SA20-3_100ns/step9_1.gro')
print(mix1_1)
#laup_1 = md.load(filedir+'LAUP_NaCl0.4_GMX/SA20-1_100ns/step9_1.trr', top=filedir+'LAUP_NaCl0.4_GMX/SA20-1_100ns/step9_1.gro')

In [ ]:
mix1_2 = md.load(filedir+'MIX1_NaCl0.4_GMX/SA20_20ns/step9_1.trr', top=filedir+'MIX1_NaCl0.4_GMX/SA20_20ns/step9_1.gro')
#laup_2 = md.load(['/gpfs/amarolab/abbyflabby/LAU/LAUP_NaCl0.4/SA20/charmm-gui/namd/step7.1_production.dcd', '/gpfs/amarolab/abbyflabby/LAU/LAUP_NaCl0.4/SA20/charmm-gui/namd/step7.2_production.dcd'], top='/gpfs/amarolab/abbyflabby/LAU/LAUP_NaCl0.4/SA20/charmm-gui/step5_assembly.xplor_ext.psf')
print(mix1_2)
#print(laup_2)

In [ ]:
mix1_3 = md.load(filedir+'MIX1_NaCl0.4_GMX/SA20-2_20ns/step9_1.trr', top=filedir+'MIX1_NaCl0.4_GMX/SA20_20ns/step9_1.gro')

In [ ]:
filedir1 = '/gpfs/amarolab/abbyflabby/'
mix1_4 = md.load(filedir1+'MIX/MIX1234_NaCl0.4_NAMD/SA20/charmm-gui/namd/step7.1_production.dcd', top=filedir1+'MIX/MIX1234_NaCl0.4_NAMD/SA20/charmm-gui/step5_assembly.xplor_ext.psf')

In [ ]:
dcds_mix1_5 = [filedir1+'MIX/MIX1234_NaCl0.4_NAMD/SA20-2/charmm-gui/namd/step7.2_production.dcd', filedir1+'MIX/MIX1234_NaCl0.4_NAMD/SA20-2/charmm-gui/namd/step7.3_production.inp.dcd', filedir1+'MIX/MIX1234_NaCl0.4_NAMD/SA20-2/charmm-gui/namd/step7.31_production.inp.dcd', filedir1+'MIX/MIX1234_NaCl0.4_NAMD/SA20-2/charmm-gui/namd/step7.32_production.inp.dcd', filedir1+'MIX/MIX1234_NaCl0.4_NAMD/SA20-2/charmm-gui/namd/step7.33_production.dcd']
mix1_5 = md.load(dcds_mix1_5, top=filedir1+'MIX/MIX1234_NaCl0.4_NAMD/SA20/charmm-gui/step5_assembly.xplor_ext.psf')     


In [ ]:
dcds_mix1_6 = [filedir1+'MIX/MIX1234_NaCl0.4_NAMD/SA20-3/charmm-gui/namd/step7.1_production.dcd', 
               filedir1+'MIX/MIX1234_NaCl0.4_NAMD/SA20-3/charmm-gui/namd/step7.2_production.dcd', 
               filedir1+'MIX/MIX1234_NaCl0.4_NAMD/SA20-3/charmm-gui/namd/step7.3_production.inp.dcd', 
               filedir1+'MIX/MIX1234_NaCl0.4_NAMD/SA20-3/charmm-gui/namd/step7.31_production.inp.dcd', 
               filedir1+'MIX/MIX1234_NaCl0.4_NAMD/SA20-3/charmm-gui/namd/step7.32_production.inp.dcd', 
               filedir1+'MIX/MIX1234_NaCl0.4_NAMD/SA20-3/charmm-gui/namd/step7.33_production.dcd',
               filedir1+'MIX/MIX1234_NaCl0.4_NAMD/SA20-3/charmm-gui/namd/step7.34_production.dcd',
               filedir1+'MIX/MIX1234_NaCl0.4_NAMD/SA20-3/charmm-gui/namd/step7.35_production.dcd']
mix1_6 = md.load(dcds_mix1_5, top=filedir1+'MIX/MIX1234_NaCl0.4_NAMD/SA20-3/charmm-gui/step5_assembly.xplor_ext.psf') 

In [9]:
#def get_nematic_orders(traj):
#    traj.center_coordinates()
#    residues = [residue.index for residue in traj.top.residues if residue.name =='PALP' or residue.name == 'MYRP' or residue.name=='LAUP' or residue.name=='STEP']
#    atoms = []
#    for i in residues:
#        atoms.append([atom.index for atom in traj.top.atoms if atom.residue.index == residues[i] and traj.xyz[0][atom.index][2]<0 and atom.element.symbol == 'C'])
#    return(md.compute_nematic_order(traj, indices=atoms))
#no6 = get_nematic_orders(mix1_6)
#no7 = get_nematic_orders(mix1_5)
#no8 = get_nematic_orders(mix1_4)
#no9 = get_nematic_orders(mix1_3)
#no10 = get_nematic_orders(mix1_2)
#no11 = get_nematic_orders(mix1_1)

In [10]:
#print(np.mean(no6))
#print(np.mean(no7))
#print(np.mean(no8))
#print(np.mean(no9))
#print(np.mean(no10))
#print(np.mean(no11))

In [5]:
def get_nematic_orders_upper_chains(traj):
    traj.center_coordinates()
    residues = [residue.index for residue in traj.top.residues if residue.name =='PALP' or residue.name == 'MYRP' or residue.name=='LAUP' or residue.name=='STEP']
    atoms = []
    for i in residues:
        atoms.append([atom.index for atom in traj.top.atoms if atom.residue.index == residues[i] and traj.xyz[0][atom.index][2]>0])
    return(md.compute_nematic_order(traj, indices=atoms))
def get_nematic_orders_lower_chains(traj):
    traj.center_coordinates()
    residues = [residue.index for residue in traj.top.residues if residue.name =='PALP' or residue.name == 'MYRP' or residue.name=='LAUP' or residue.name=='STEP']
    atoms = []
    for i in residues:
        atoms.append([atom.index for atom in traj.top.atoms if atom.residue.index == residues[i] and traj.xyz[0][atom.index][2]<=0])
    return(md.compute_nematic_order(traj, indices=atoms))
#no1u = get_nematic_orders_upper_chains(mix1_6)
#no2u = get_nematic_orders_upper_chains(mix1_5)
#no3u = get_nematic_orders_upper_chains(mix1_4)
#no4u = get_nematic_orders_upper_chains(mix1_3)
#no50u = get_nematic_orders_upper_chains(mix1_2)
#no6u = get_nematic_orders_upper_chains(mix1_1)
#no1l = get_nematic_orders_upper_chains(mix1_6)
#no2l = get_nematic_orders_upper_chains(mix1_5)
#no3l = get_nematic_orders_upper_chains(mix1_4)
#no4l = get_nematic_orders_upper_chains(mix1_3)
#no5l = get_nematic_orders_upper_chains(mix1_2)
#no6l = get_nematic_orders_upper_chains(mix1_1)
no1u = get_nematic_orders_upper_chains(laup_1)
no2u = get_nematic_orders_upper_chains(laup_2)
no1l = get_nematic_orders_lower_chains(laup_1)
no2l = get_nematic_orders_lower_chains(laup_2)



In [7]:
#print(np.mean(no6C))
#print(np.mean(no7C))
#print(np.mean(no8C))
#print(np.mean(no9C))
#print(np.mean(no10C))
#print(np.mean(no11C))
print(np.mean(no1u))
print(np.mean(no2u))
print(np.mean(no1l))
print(np.mean(no2l))

0.3204392207952497
0.3173545242089625
0.38696601357712523
0.29752235660322296


In [8]:
datas = np.array([no1u, no2u, no1l, no2l])

In [11]:
def writeToCSV(data):
    l = len(data)
    for i, d in enumerate(data):
        with open('/gpfs/amarolab/monolayers/Monolayer_Sims_Raw_Data/LAUP_NaCl0.4_SA20_pH2/NematicOrder_'+str(i+1)+'.csv','w') as csv:
            for dat in d:
                csv.write(str(dat)+"\n")
writeToCSV(datas)